In [1]:
import requests
import pandas as pd
import numpy as np
import time
import copy
from tqdm import tqdm
import os
from pathlib import Path

# In phiên bản để debug
print("pandas", pd.__version__, "| numpy", np.__version__)

pandas 2.3.3 | numpy 1.26.4


In [6]:
pip install geopy


  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached geographiclib-2.1-py3-none-any.whl.metadata (1.6 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached geographiclib-2.1-py3-none-any.whl (40 kB)

   -------------------- ------------------- 1/2 [geopy]
   ---------------------------------------- 2/2 [geopy]

Note: you may need to restart the kernel to use updated packages.


In [15]:
import datetime
from geopy.distance import great_circle
import copy # Cần thiết để sao chép waypoints
from tqdm import tqdm # Import thư viện tqdm

ROUTE_CONFIG = {
    'VNSGN-CNSHA': {
        'waypoints': [
            {'name': 'Cang TP.HCM (Start)', 'lat': 10.75, 'lon': 106.70, 'time_hr': 0.0},
            {'name': 'Vung Tau Exit', 'lat': 10.30, 'lon': 107.10, 'time_hr': 1.5},
            {'name': 'Nam Bien Dong', 'lat': 13.00, 'lon': 110.00, 'time_hr': 15.0},
            {'name': 'Gan Hoang Sa', 'lat': 16.00, 'lon': 112.00, 'time_hr': 25.0},
            {'name': 'Gan Dao Hai Nam', 'lat': 20.00, 'lon': 113.00, 'time_hr': 45.0},
            {'name': 'Eo Bien Luzon/Taiwan', 'lat': 23.00, 'lon': 118.00, 'time_hr': 70.0},
            {'name': 'Gan Ninh Ba', 'lat': 29.00, 'lon': 122.00, 'time_hr': 85.0},
            {'name': 'Cang Thuong Hai (End)', 'lat': 31.23, 'lon': 121.50, 'time_hr': 88.0} # Steaming time: ~3.7 ngày
        ],
        'duration': (168, 216) # Thời gian thực tế: 7-9 ngày
    },
    'VNHPH-CNQZH': {
        'waypoints': [
    {'name': 'T+0h (Depart Haiphong)', 'lat': 20.842740, 'lon': 106.772600, 'time_hr': 0.00},
    {'name': 'T+2h', 'lat': 20.942682, 'lon': 106.992011, 'time_hr': 2.00},
    {'name': 'T+4h', 'lat': 21.042344, 'lon': 107.211715, 'time_hr': 4.00},
    {'name': 'T+6h', 'lat': 21.141723, 'lon': 107.431713, 'time_hr': 6.00},
    {'name': 'T+8h', 'lat': 21.240817, 'lon': 107.652006, 'time_hr': 8.00},
    {'name': 'T+10h','lat': 21.339625, 'lon': 107.872594, 'time_hr': 10.00},
    {'name': 'T+12h','lat': 21.438144, 'lon': 108.093480, 'time_hr': 12.00},
    {'name': 'T+14h','lat': 21.536374, 'lon': 108.314663, 'time_hr': 14.00},
    {'name': 'T+16h','lat': 21.634311, 'lon': 108.536146, 'time_hr': 16.00},
    {'name': 'T+17h (Arrive Qinzhou)',   'lat': 21.683170, 'lon': 108.647000, 'time_hr': 17.00}
]
,
        'duration': (24, 32) # Thời gian thực tế (bao gồm chờ tàu): 1-2 ngày
    },
    'VNSGN-KRPUS': {
        'waypoints': [
{'name': 'Ho Chi Minh Port (Start)', 'lat': 10.77, 'lon': 106.71, 'time_hr': 0.0},
{'name': 'Vung Tau Exit', 'lat': 10.33, 'lon': 107.07, 'time_hr': 2.2},
{'name': 'South China Sea Entry', 'lat': 10.5, 'lon': 108.0, 'time_hr': 5.9},
{'name': 'South China Sea Mid South', 'lat': 13.0, 'lon': 110.0, 'time_hr': 18.7},
{'name': 'South China Sea Central', 'lat': 16.0, 'lon': 112.0, 'time_hr': 33.1},
{'name': 'South China Sea North', 'lat': 19.0, 'lon': 113.5, 'time_hr': 46.6},
{'name': 'Hainan East Approach', 'lat': 20.0, 'lon': 114.0, 'time_hr': 51.1},
{'name': 'Taiwan Strait Entry', 'lat': 23.0, 'lon': 117.0, 'time_hr': 68.1},
{'name': 'Taiwan Strait Mid', 'lat': 24.0, 'lon': 118.0, 'time_hr': 73.7},
{'name': 'Taiwan Strait Exit', 'lat': 25.0, 'lon': 119.5, 'time_hr': 80.9},
{'name': 'East China Sea South', 'lat': 27.0, 'lon': 121.0, 'time_hr': 90.9},
{'name': 'East China Sea Mid', 'lat': 30.0, 'lon': 123.0, 'time_hr': 105.3},
{'name': 'East China Sea North', 'lat': 33.0, 'lon': 126.0, 'time_hr': 122.3},
{'name': 'Busan Approach', 'lat': 34.5, 'lon': 128.0, 'time_hr': 132.3},
{'name': 'Busan Port (End)', 'lat': 35.1, 'lon': 129.05, 'time_hr': 137.1}
],
        'duration': (168, 216) # Thời gian thực tế: 7-9 ngày
    },

    # --- CÁC TUYẾN CẦN CẬP NHẬT DỮ LIỆU ---
    # [PLACEHOLDER] - Vui lòng thay thế waypoints và duration bằng dữ liệu thực tế
    'VNSGN-USMIA': {
        'waypoints': [
    {"name": "Hai Phong Port (Start)", "lat": 20.86, "lon": 106.68, "time_hr": 0.0},
    {"name": "Gulf of Tonkin Exit", "lat": 19.50, "lon": 108.50, "time_hr": 12.0},
    {"name": "South China Sea Central", "lat": 15.00, "lon": 112.00, "time_hr": 48.0},
    {"name": "Singapore Strait", "lat": 1.27, "lon": 103.83, "time_hr": 120.0},
    {"name": "Malacca Strait Exit", "lat": 5.00, "lon": 98.00, "time_hr": 168.0},
    {"name": "Indian Ocean Crossing", "lat": 0.00, "lon": 80.00, "time_hr": 360.0},
    {"name": "Gulf of Aden", "lat": 12.00, "lon": 45.00, "time_hr": 480.0},
    {"name": "Suez Canal Entry", "lat": 29.93, "lon": 32.55, "time_hr": 540.0},
    {"name": "Mediterranean Sea", "lat": 35.00, "lon": 20.00, "time_hr": 600.0},
    {"name": "Gibraltar Strait", "lat": 36.00, "lon": -5.00, "time_hr": 660.0},
    {"name": "Atlantic Ocean Crossing", "lat": 30.00, "lon": -40.00, "time_hr": 840.0},
    {"name": "Caribbean Sea Approach", "lat": 20.00, "lon": -75.00, "time_hr": 960.0},
    {"name": "Miami Port (End)", "lat": 25.77, "lon": -80.20, "time_hr": 1008.0}
],
        'duration': (1024, 1080) # Thời gian thực tế: 38-45 ngày
    },

    'VNHPH-CNSHA': {
        'waypoints': [
    {'name': 'T+0h (Depart Hai Phong)', 'lat': 20.842740, 'lon': 106.772600, 'time_hr': 0.00},
    {'name': 'T+8h',                       'lat': 21.910308, 'lon': 108.093630, 'time_hr': 8.00},
    {'name': 'T+16h',                      'lat': 22.967173, 'lon': 109.434508, 'time_hr': 16.00},
    {'name': 'T+24h',                      'lat': 24.012587, 'lon': 110.796402, 'time_hr': 24.00},
    {'name': 'T+32h',                      'lat': 25.045772, 'lon': 112.180485, 'time_hr': 32.00},
    {'name': 'T+40h',                      'lat': 26.065916, 'lon': 113.587942, 'time_hr': 40.00},
    {'name': 'T+48h',                      'lat': 27.072176, 'lon': 115.019961, 'time_hr': 48.00},
    {'name': 'T+56h',                      'lat': 28.063671, 'lon': 116.477730, 'time_hr': 56.00},
    {'name': 'T+64h',                      'lat': 29.039483, 'lon': 117.962430, 'time_hr': 64.00},
    {'name': 'T+72h',                      'lat': 29.998659, 'lon': 119.475233, 'time_hr': 72.00},
    {'name': 'T+80h',                      'lat': 30.940204, 'lon': 121.017287, 'time_hr': 80.00},
    {'name': 'T+82.41h (Arrive Shanghai)', 'lat': 31.220234, 'lon': 121.487734, 'time_hr': 82.41}
],
        'duration': (120, 168) # Thời gian thực tế: 5-7 ngày
    },
    # [PLACEHOLDER] - USCHI (Chicago) là cảng nội địa. Giả định đi đến USLGB (Los Angeles)
    'VNSGN-USCHI': {
        'waypoints': [
    {"name": "Ho Chi Minh Port (Start)", "lat": 10.75, "lon": 106.70, "time_hr": 0.0},
    {"name": "Vung Tau Exit", "lat": 10.30, "lon": 107.10, "time_hr": 4.0},
    {"name": "South China Sea", "lat": 15.00, "lon": 112.00, "time_hr": 36.0},
    {"name": "Taiwan Strait", "lat": 24.00, "lon": 120.00, "time_hr": 120.0},
    {"name": "East China Sea", "lat": 30.00, "lon": 125.00, "time_hr": 192.0},
    {"name": "Pacific Ocean Crossing", "lat": 35.00, "lon": -160.00, "time_hr": 576.0},
    {"name": "West Coast Approach", "lat": 40.00, "lon": -125.00, "time_hr": 768.0},
    {"name": "San Francisco Bay", "lat": 37.77, "lon": -122.42, "time_hr": 816.0},
    {"name": "Panama Canal Entry", "lat": 9.38, "lon": -79.92, "time_hr": 1104.0},
    {"name": "Gulf of Mexico", "lat": 25.00, "lon": -90.00, "time_hr": 1248.0},
    {"name": "New Orleans Transfer", "lat": 29.95, "lon": -90.07, "time_hr": 1296.0},
    {"name": "Mississippi River System", "lat": 35.00, "lon": -90.00, "time_hr": 1440.0},
    {"name": "Illinois Waterway", "lat": 40.00, "lon": -88.00, "time_hr": 1560.0},
    {"name": "Chicago Port (End)", "lat": 41.88, "lon": -87.62, "time_hr": 1608.0}
],
        'duration': (1500, 1550, 1450) # Thời gian thực tế (chỉ đường biển): 20-24 ngày
    },
    # [PLACEHOLDER]
    'VNHPH-USMIA': {
        'waypoints': [
    {"name": "Hai Phong Port (Start)", "lat": 20.86, "lon": 106.68, "time_hr": 0.0},
    {"name": "Gulf of Tonkin Exit", "lat": 19.50, "lon": 108.50, "time_hr": 12.0},
    {"name": "South China Sea Central", "lat": 15.00, "lon": 112.00, "time_hr": 48.0},
    {"name": "Singapore Strait", "lat": 1.27, "lon": 103.83, "time_hr": 120.0},
    {"name": "Malacca Strait Exit", "lat": 5.00, "lon": 98.00, "time_hr": 168.0},
    {"name": "Indian Ocean Crossing", "lat": 0.00, "lon": 80.00, "time_hr": 360.0},
    {"name": "Gulf of Aden", "lat": 12.00, "lon": 45.00, "time_hr": 480.0},
    {"name": "Suez Canal Entry", "lat": 29.93, "lon": 32.55, "time_hr": 540.0},
    {"name": "Mediterranean Sea", "lat": 35.00, "lon": 20.00, "time_hr": 600.0},
    {"name": "Gibraltar Strait", "lat": 36.00, "lon": -5.00, "time_hr": 660.0},
    {"name": "Atlantic Ocean Crossing", "lat": 30.00, "lon": -40.00, "time_hr": 840.0},
    {"name": "Caribbean Sea Approach", "lat": 20.00, "lon": -75.00, "time_hr": 960.0},
    {"name": "Miami Port (End)", "lat": 25.77, "lon": -80.20, "time_hr": 1008.0}
],
        'duration': (960, 1128) # Thời gian thực tế: 40-47 ngày
    },
    # [PLACEHOLDER]
    'VNSGN-KRINC': {
        'waypoints': [
{'name': 'Ho Chi Minh Port (Start)', 'lat': 10.77, 'lon': 106.71, 'time_hr': 0.0},
{'name': 'Vung Tau Exit', 'lat': 10.33, 'lon': 107.07, 'time_hr': 2.2},
{'name': 'South China Sea Entry', 'lat': 10.5, 'lon': 108.0, 'time_hr': 5.9},
{'name': 'South China Sea Mid South', 'lat': 13.0, 'lon': 110.0, 'time_hr': 18.7},
{'name': 'South China Sea Central', 'lat': 16.0, 'lon': 112.0, 'time_hr': 33.1},
{'name': 'South China Sea North', 'lat': 19.0, 'lon': 113.5, 'time_hr': 46.6},
{'name': 'Hainan East Approach', 'lat': 20.0, 'lon': 114.0, 'time_hr': 51.1},
{'name': 'Taiwan Strait Entry', 'lat': 23.0, 'lon': 117.0, 'time_hr': 68.1},
{'name': 'Taiwan Strait Mid', 'lat': 24.0, 'lon': 118.0, 'time_hr': 73.7},
{'name': 'Taiwan Strait Exit', 'lat': 25.0, 'lon': 119.5, 'time_hr': 80.9},
{'name': 'East China Sea South', 'lat': 27.0, 'lon': 121.0, 'time_hr': 90.9},
{'name': 'East China Sea Mid', 'lat': 30.0, 'lon': 123.0, 'time_hr': 105.3},
{'name': 'East China Sea North', 'lat': 33.0, 'lon': 126.0, 'time_hr': 122.3},
{'name': 'Yellow Sea South', 'lat': 34.0, 'lon': 126.5, 'time_hr': 126.2},
{'name': 'Yellow Sea Mid', 'lat': 35.0, 'lon': 126.0, 'time_hr': 131.2},
{'name': 'Incheon Approach', 'lat': 37.0, 'lon': 126.5, 'time_hr': 141.2},
{'name': 'Incheon Port (End)', 'lat': 37.46, 'lon': 126.64, 'time_hr': 143.3}
],
        'duration': (192, 240) # Thời gian thực tế: 8-10 ngày
    },
    # [PLACEHOLDER]
    'VNDAD-KRPUS': {
        'waypoints': [
{'name': 'Da Nang Port (Start)', 'lat': 16.05, 'lon': 108.22, 'time_hr': 0.0},
{'name': 'Da Nang Channel Exit', 'lat': 15.8, 'lon': 108.5, 'time_hr': 1.3},
{'name': 'South China Sea Entry', 'lat': 16.0, 'lon': 110.0, 'time_hr': 5.6},
{'name': 'South China Sea Central', 'lat': 18.0, 'lon': 112.0, 'time_hr': 15.0},
{'name': 'South China Sea North', 'lat': 21.0, 'lon': 113.0, 'time_hr': 25.0},
{'name': 'Hainan Approach', 'lat': 20.0, 'lon': 110.5, 'time_hr': 23.1},
{'name': 'Taiwan Strait Entry', 'lat': 23.0, 'lon': 117.0, 'time_hr': 40.6},
{'name': 'Taiwan Strait Mid', 'lat': 24.0, 'lon': 118.0, 'time_hr': 46.2},
{'name': 'Taiwan Strait Exit', 'lat': 25.0, 'lon': 119.5, 'time_hr': 53.4},
{'name': 'East China Sea South', 'lat': 27.0, 'lon': 121.0, 'time_hr': 63.4},
{'name': 'East China Sea Mid', 'lat': 30.0, 'lon': 123.0, 'time_hr': 77.8},
{'name': 'East China Sea North', 'lat': 33.0, 'lon': 126.0, 'time_hr': 91.9},
{'name': 'Busan Approach', 'lat': 34.5, 'lon': 128.0, 'time_hr': 101.9},
{'name': 'Busan Port (End)', 'lat': 35.1, 'lon': 129.05, 'time_hr': 105.6}
],
        'duration': (120, 168) # Thời gian thực tế: 5-7 ngày
    }
}


# --- 2. Thông số Mô phỏng ---
GPS_NOISE_LEVEL = 0.005
TIME_JITTER_PERCENT = 0.05
INPUT_CSV_FILE = r"C:\Users\Dell\Downloads\12k_trip_part_2-2.csv"
OUTPUT_CSV_FILE = r"C:\Users\Dell\Downloads\gps_trips_part2_2.csv"

# --- 3. Hàm Tính toán (Giữ nguyên) ---
def calculate_distance_nm(row):
    """Hàm tính khoảng cách great-circle (hải lý)"""
    if pd.isna(row['prev_lat']):
        return 0.0
    point_a = (row['prev_lat'], row['prev_lon'])
    point_b = (row['latitude'], row['longitude'])
    return great_circle(point_a, point_b).nautical

# --- 4. HÀM MÔ PHỎNG (ĐÃ CẬP NHẬT) ---
def simulate_trip(trip_id, fruit_cate, start_time, base_waypoints, realistic_duration_hours, route_key):
    """
    Mô phỏng toàn bộ dữ liệu GPS cho một chuyến đi duy nhất.
    Nhận vào waypoints cơ sở VÀ khoảng thời gian thực tế.
    """

    # SỬA LỖI: Sao chép lộ trình GỐC trước khi thực hiện bất kỳ thay đổi nào
    trip_waypoints = copy.deepcopy(base_waypoints)

    # 4.0. CO GIÃN THỜI GIAN (SLOW STEAMING)
    # Lấy tổng thời gian gốc (ví dụ: 88.0 giờ)
    original_total_time = trip_waypoints[-1]['time_hr']

    # Nếu thời gian gốc = 0 (lỗi dữ liệu), thoát sớm
    if original_total_time == 0:
        print(f"  > LỖI: {trip_id} có lộ trình gốc với thời gian = 0. Bỏ qua.")
        return pd.DataFrame() # Trả về DataFrame rỗng

    # Chọn ngẫu nhiên một tổng thời gian thực tế (ví dụ: 170.5 giờ)
    simulated_total_time = np.random.uniform(
        realistic_duration_hours[0],
        realistic_duration_hours[1]
    )

    # Tính toán tỷ lệ co giãn (ví dụ: 170.5 / 88.0 = 1.93)
    scale_factor = simulated_total_time / original_total_time

    # Áp dụng tỷ lệ này cho TẤT CẢ các mốc thời gian (trừ điểm 0)
    for wp in trip_waypoints:
        wp['time_hr'] = wp['time_hr'] * scale_factor

    # 4.1. TẠO NHIỄU THỜI GIAN (TIME JITTER)
    # (Không cần copy.deepcopy một lần nữa)

    # trip_key_times sẽ bị thay đổi ngẫu nhiên
    trip_key_times = [0.0] # Điểm đầu tiên luôn là 0.0

    for i in range(1, len(trip_waypoints)):
        # Lấy thời gian gốc (ĐÃ ĐƯỢC CO GIÃN)
        base_time = trip_waypoints[i]['time_hr']
        prev_base_time = trip_waypoints[i-1]['time_hr']

        # Khoảng thời gian (đã co giãn) giữa 2 điểm
        segment_duration = base_time - prev_base_time

        # Tạo nhiễu ngẫu nhiên (ví dụ: từ -5% đến +5%)
        jitter = np.random.uniform(-TIME_JITTER_PERCENT, TIME_JITTER_PERCENT)

        # Tính khoảng thời gian mới (thực tế)
        new_segment_duration = segment_duration * (1 + jitter)

        # Thời gian mới = thời gian (đã bị nhiễu) của điểm trước + khoảng thời gian mới
        new_time = trip_key_times[-1] + new_segment_duration

        # Đảm bảo thời gian luôn tăng
        if new_time <= trip_key_times[-1]:
            new_time = trip_key_times[-1] + 0.1 # Thêm 1 khoảng nhỏ

        trip_key_times.append(new_time)

    # Lấy ra các mốc vĩ độ/kinh độ (không đổi từ lộ trình gốc)
    # QUAN TRỌNG: Dùng 'base_waypoints' gốc để lấy vị trí
    key_lats = [wp['lat'] for wp in base_waypoints]
    key_lons = [wp['lon'] for wp in base_waypoints]

    # 4.2. Chạy Nội suy (Interpolation)
    total_hours = int(round(trip_key_times[-1])) # Tổng thời gian mới (đã bị nhiễu)
    target_hours = np.arange(total_hours + 1) # [0, 1, 2, ..., total_hours]

    # Nội suy Vị trí dựa trên Thời gian gốc (để tàu di chuyển đúng)
    # Nội suy Thời gian dựa trên Thời gian đã co giãn (để lấy mẫu)
    original_key_times = [wp['time_hr'] for wp in base_waypoints]

    # Chú ý: Chúng ta nội suy lat/lon dựa trên thời gian GỐC (0-88 giờ)
    # nhưng chúng ta lấy mẫu tại các giờ MỚI (0-170 giờ)
    # SỬA LỖI LOGIC: Phải nội suy lat/lon dựa trên MỐC THỜI GIAN ĐÃ CO GIÃN (trip_key_times)
    # thì vị trí mới khớp với thời gian
    interp_lats = np.interp(target_hours, trip_key_times, key_lats)
    interp_lons = np.interp(target_hours, trip_key_times, key_lons)

    # 4.3. Tạo DataFrame
    df = pd.DataFrame({
        'trip_id': trip_id,
        'fruit_cate': fruit_cate, # Sửa từ fruit_name thành fruit_cate
        'hour': target_hours,
        'latitude_ideal': interp_lats,
        'longitude_ideal': interp_lons,
        'Route': route_key
    })

    # Thêm cột timestamp thực tế
    start_time_dt = pd.to_datetime(start_time)
    df['timestamp'] = df['hour'].apply(lambda h: start_time_dt + datetime.timedelta(hours=h))

    # 4.4. Thêm Nhiễu GPS (Vị trí)
    noise_lat = np.random.normal(0, GPS_NOISE_LEVEL, size=len(df))
    noise_lon = np.random.normal(0, GPS_NOISE_LEVEL, size=len(df))
    df['latitude'] = df['latitude_ideal'] + noise_lat
    df['longitude'] = df['longitude_ideal'] + noise_lon

    # 4.5. Tính toán Tốc độ
    df['prev_lat'] = df['latitude'].shift(1)
    df['prev_lon'] = df['longitude'].shift(1)
    df['distance_nm'] = df.apply(calculate_distance_nm, axis=1)
    df['speed_knots'] = (df['distance_nm'] / 1.0).round(2) # Giả định 1 giờ

    # 4.6. Dọn dẹp
    final_columns = ['trip_id', 'fruit_cate', 'timestamp', 'latitude', 'longitude', 'speed_knots','Route']
    df_final = df[final_columns].copy()
    df_final.loc[0, 'speed_knots'] = 0.0 # Tốc độ tại điểm bắt đầu = 0

    # Print inside the function for individual trip progress (optional, tqdm is better)
    # print(f"  > Đã mô phỏng {ship_id} (Tuyến {row['Route']}), tổng thời gian: {trip_key_times[-1]:.2f} giờ ({len(df_final)} điểm dữ liệu)")

    return df_final

# --- 5. HÀM CHÍNH (ĐÃ CẬP NHẬT) ---
def main():
    print("Bắt đầu mô phỏng hàng loạt chuyến đi...")

    # 5.1. Đọc file CSV đầu vào
    try:
        journeys_df = pd.read_csv(INPUT_CSV_FILE)
    except FileNotFoundError:
        print(f"Không tìm thấy file '{INPUT_CSV_FILE}'. Tạo file mẫu...")

        # Cập nhật file mẫu để chứa cột 'Route'
        sample_data = {
            'trip_id': ['SHIP_001', 'SHIP_002', 'SHIP_003', 'SHIP_004'],
            'fruit_cate': ['Banana', 'Orange', 'Tomato', 'Orange'],
            'start_time': ['2025-10-01T08:00:00', '2025-10-01T12:00:00', '2025-10-02T01:00:00', '2025-10-02T05:00:00'],
            'Route': ['VNSGN-CNSHA', 'VNHPH-CNQZH', 'VNSGN-KRPUS', 'VNSGN-USMIA'] # Thêm cột Route
        }
        sample_df = pd.DataFrame(sample_data)
        sample_df.to_csv(INPUT_CSV_FILE, index=False)

        print(f"Đã tạo file '{INPUT_CSV_FILE}' với dữ liệu mẫu (bao gồm cột 'Route'). Vui lòng chạy lại script.")
        return # Dừng lại để người dùng xem file mẫu

    print(f"Đã đọc {len(journeys_df)} chuyến đi từ {INPUT_CSV_FILE}.")
    if 'Route' not in journeys_df.columns:
        print(f"LỖI: File '{INPUT_CSV_FILE}' của bạn bị thiếu cột 'Route'.")
        print("Vui lòng thêm cột 'Route' (ví dụ: 'VNSGN-CNSHA', 'VNHPH-CNQZH') và chạy lại.")
        return

    # 5.2. Chạy mô phỏng cho từng chuyến
    all_trips_data = [] # List để chứa các DataFrame của mỗi chuyến

    # Sử dụng tqdm để bọc vòng lặp qua các hàng của journeys_df
    for index, row in tqdm(journeys_df.iterrows(), total=len(journeys_df), desc="Đang mô phỏng chuyến đi"):
        # Loại bỏ khoảng trắng ở đầu/cuối VÀ thay thế tất cả khoảng trắng ở giữa
        route_key = row['Route'].replace(" ", "").strip()

        try:
            # Tra cứu cấu hình (waypoints và duration) dựa trên 'Route'
            config = ROUTE_CONFIG[route_key]

            trip_df = simulate_trip(
                trip_id=row['trip_id'],
                fruit_cate=row['fruit_cate'],
                start_time=row['start_time'],
                base_waypoints=config['waypoints'], # Cung cấp waypoints cụ thể
                realistic_duration_hours=config['duration'], # Cung cấp thời gian cụ thể
                route_key=route_key
            )
            if not trip_df.empty: # Chỉ thêm nếu DataFrame không rỗng (trường hợp lỗi thời gian = 0)
                all_trips_data.append(trip_df)

        except KeyError:
            # Bỏ qua nếu không tìm thấy route_key trong ROUTE_CONFIG
            print(f"\n  > CẢNH BÁO: Không tìm thấy cấu hình cho Route '{route_key}' (trip_id: {row['trip_id']}). Bỏ qua.") # Sửa ship_id thành trip_id
            continue
        #except Exception as e:
            #print(f"\n  > LỖI khi xử lý {row['trip_id']} (Route {route_key}): {e}")
            #continue

    # 5.3. Kết hợp và Lưu kết quả
    if not all_trips_data:
        print("\nKhông có dữ liệu nào được mô phỏng thành công.")
        return

    # Ghép tất cả các DataFrame trong list lại thành 1 DataFrame lớn
    final_combined_df = pd.concat(all_trips_data, ignore_index=True)

    # Sắp xếp theo tàu và thời gian cho dễ đọc (tùy chọn)
    final_combined_df = final_combined_df.sort_values(by=['trip_id', 'timestamp'])

    # Lưu ra file CSV tổng
    final_combined_df.to_csv(OUTPUT_CSV_FILE, index=False, encoding='utf-8-sig')

    print(f"\n--- HOÀN TẤT ---")
    print(f"Đã lưu tổng cộng {len(final_combined_df)} điểm dữ liệu của {len(all_trips_data)} chuyến đi vào file:")
    print(f"{OUTPUT_CSV_FILE}")

# --- 6. Thực thi ---
if __name__ == "__main__":
    main()

Bắt đầu mô phỏng hàng loạt chuyến đi...
Đã đọc 1500 chuyến đi từ C:\Users\Dell\Downloads\12k_trip_part_2-2.csv.


Đang mô phỏng chuyến đi: 100%|██████████| 1500/1500 [00:20<00:00, 73.38it/s]



--- HOÀN TẤT ---
Đã lưu tổng cộng 734320 điểm dữ liệu của 1500 chuyến đi vào file:
C:\Users\Dell\Downloads\gps_trips_part2_2.csv


# 1. Định nghĩa ROUTE_CONFIG (waypoints + duration)

In [2]:
ROUTE_CONFIG = {
    'VNSGN-CNSHA': {
        'waypoints': [
            {'name': 'Cang TP.HCM (Start)', 'lat': 10.75, 'lon': 106.70, 'time_hr': 0.0},
            {'name': 'Vung Tau Exit', 'lat': 10.30, 'lon': 107.10, 'time_hr': 1.5},
            {'name': 'Nam Bien Dong', 'lat': 13.00, 'lon': 110.00, 'time_hr': 15.0},
            {'name': 'Gan Hoang Sa', 'lat': 16.00, 'lon': 112.00, 'time_hr': 25.0},
            {'name': 'Gan Dao Hai Nam', 'lat': 20.00, 'lon': 113.00, 'time_hr': 45.0},
            {'name': 'Eo Bien Luzon/Taiwan', 'lat': 23.00, 'lon': 118.00, 'time_hr': 70.0},
            {'name': 'Gan Ninh Ba', 'lat': 29.00, 'lon': 122.00, 'time_hr': 85.0},
            {'name': 'Cang Thuong Hai (End)', 'lat': 31.23, 'lon': 121.50, 'time_hr': 88.0}
        ],
        'duration': (168, 216)
    },

    'VNHPH-CNQZH': {
        'waypoints': [
            {'name': 'T+0h (Depart Haiphong)', 'lat': 20.842740, 'lon': 106.772600, 'time_hr': 0.00},
            {'name': 'T+2h', 'lat': 20.942682, 'lon': 106.992011, 'time_hr': 2.00},
            {'name': 'T+4h', 'lat': 21.042344, 'lon': 107.211715, 'time_hr': 4.00},
            {'name': 'T+6h', 'lat': 21.141723, 'lon': 107.431713, 'time_hr': 6.00},
            {'name': 'T+8h', 'lat': 21.240817, 'lon': 107.652006, 'time_hr': 8.00},
            {'name': 'T+10h', 'lat': 21.339625, 'lon': 107.872594, 'time_hr': 10.00},
            {'name': 'T+12h', 'lat': 21.438144, 'lon': 108.093480, 'time_hr': 12.00},
            {'name': 'T+14h', 'lat': 21.536374, 'lon': 108.314663, 'time_hr': 14.00},
            {'name': 'T+16h', 'lat': 21.634311, 'lon': 108.536146, 'time_hr': 16.00},
            {'name': 'T+17h (Arrive Qinzhou)', 'lat': 21.683170, 'lon': 108.647000, 'time_hr': 17.00}
        ],
        'duration': (24, 32)
    },

    'VNSGN-KRPUS': {
        'waypoints': [
            {'name': 'Ho Chi Minh Port (Start)', 'lat': 10.77, 'lon': 106.71, 'time_hr': 0.0},
            {'name': 'Vung Tau Exit', 'lat': 10.33, 'lon': 107.07, 'time_hr': 2.2},
            {'name': 'South China Sea Entry', 'lat': 10.5, 'lon': 108.0, 'time_hr': 5.9},
            {'name': 'South China Sea Mid South', 'lat': 13.0, 'lon': 110.0, 'time_hr': 18.7},
            {'name': 'South China Sea Central', 'lat': 16.0, 'lon': 112.0, 'time_hr': 33.1},
            {'name': 'South China Sea North', 'lat': 19.0, 'lon': 113.5, 'time_hr': 46.6},
            {'name': 'Hainan East Approach', 'lat': 20.0, 'lon': 114.0, 'time_hr': 51.1},
            {'name': 'Taiwan Strait Entry', 'lat': 23.0, 'lon': 117.0, 'time_hr': 68.1},
            {'name': 'Taiwan Strait Mid', 'lat': 24.0, 'lon': 118.0, 'time_hr': 73.7},
            {'name': 'Taiwan Strait Exit', 'lat': 25.0, 'lon': 119.5, 'time_hr': 80.9},
            {'name': 'East China Sea South', 'lat': 27.0, 'lon': 121.0, 'time_hr': 90.9},
            {'name': 'East China Sea Mid', 'lat': 30.0, 'lon': 123.0, 'time_hr': 105.3},
            {'name': 'East China Sea North', 'lat': 33.0, 'lon': 126.0, 'time_hr': 122.3},
            {'name': 'Busan Approach', 'lat': 34.5, 'lon': 128.0, 'time_hr': 132.3},
            {'name': 'Busan Port (End)', 'lat': 35.1, 'lon': 129.05, 'time_hr': 137.1}
        ],
        'duration': (168, 216)
    },

    'VNSGN-USMIA': {
        'waypoints': [
            {'name': 'Hai Phong Port (Start)', 'lat': 20.86, 'lon': 106.68, 'time_hr': 0.0},
            {'name': 'Gulf of Tonkin Exit', 'lat': 19.50, 'lon': 108.50, 'time_hr': 12.0},
            {'name': 'South China Sea Central', 'lat': 15.00, 'lon': 112.00, 'time_hr': 48.0},
            {'name': 'Singapore Strait', 'lat': 1.27, 'lon': 103.83, 'time_hr': 120.0},
            {'name': 'Malacca Strait Exit', 'lat': 5.00, 'lon': 98.00, 'time_hr': 168.0},
            {'name': 'Indian Ocean Crossing', 'lat': 0.00, 'lon': 80.00, 'time_hr': 360.0},
            {'name': 'Gulf of Aden', 'lat': 12.00, 'lon': 45.00, 'time_hr': 480.0},
            {'name': 'Suez Canal Entry', 'lat': 29.93, 'lon': 32.55, 'time_hr': 540.0},
            {'name': 'Mediterranean Sea', 'lat': 35.00, 'lon': 20.00, 'time_hr': 600.0},
            {'name': 'Gibraltar Strait', 'lat': 36.00, 'lon': -5.00, 'time_hr': 660.0},
            {'name': 'Atlantic Ocean Crossing', 'lat': 30.00, 'lon': -40.00, 'time_hr': 840.0},
            {'name': 'Caribbean Sea Approach', 'lat': 20.00, 'lon': -75.00, 'time_hr': 960.0},
            {'name': 'Miami Port (End)', 'lat': 25.77, 'lon': -80.20, 'time_hr': 1008.0}
        ],
        'duration': (1024, 1080)
    },

    'VNHPH-CNSHA': {
        'waypoints': [
            {'name': 'T+0h (Depart Hai Phong)', 'lat': 20.842740, 'lon': 106.772600, 'time_hr': 0.00},
            {'name': 'T+8h', 'lat': 21.910308, 'lon': 108.093630, 'time_hr': 8.00},
            {'name': 'T+16h', 'lat': 22.967173, 'lon': 109.434508, 'time_hr': 16.00},
            {'name': 'T+24h', 'lat': 24.012587, 'lon': 110.796402, 'time_hr': 24.00},
            {'name': 'T+32h', 'lat': 25.045772, 'lon': 112.180485, 'time_hr': 32.00},
            {'name': 'T+40h', 'lat': 26.065916, 'lon': 113.587942, 'time_hr': 40.00},
            {'name': 'T+48h', 'lat': 27.072176, 'lon': 115.019961, 'time_hr': 48.00},
            {'name': 'T+56h', 'lat': 28.063671, 'lon': 116.477730, 'time_hr': 56.00},
            {'name': 'T+64h', 'lat': 29.039483, 'lon': 117.962430, 'time_hr': 64.00},
            {'name': 'T+72h', 'lat': 29.998659, 'lon': 119.475233, 'time_hr': 72.00},
            {'name': 'T+80h', 'lat': 30.940204, 'lon': 121.017287, 'time_hr': 80.00},
            {'name': 'T+82.41h (Arrive Shanghai)', 'lat': 31.220234, 'lon': 121.487734, 'time_hr': 82.41}
        ],
        'duration': (120, 168)
    },

    'VNSGN-USCHI': {
        'waypoints': [
            {'name': 'Ho Chi Minh Port (Start)', 'lat': 10.75, 'lon': 106.70, 'time_hr': 0.0},
            {'name': 'Vung Tau Exit', 'lat': 10.30, 'lon': 107.10, 'time_hr': 4.0},
            {'name': 'South China Sea', 'lat': 15.00, 'lon': 112.00, 'time_hr': 36.0},
            {'name': 'Taiwan Strait', 'lat': 24.00, 'lon': 120.00, 'time_hr': 120.0},
            {'name': 'East China Sea', 'lat': 30.00, 'lon': 125.00, 'time_hr': 192.0},
            {'name': 'Pacific Ocean Crossing', 'lat': 35.00, 'lon': -160.00, 'time_hr': 576.0},
            {'name': 'West Coast Approach', 'lat': 40.00, 'lon': -125.00, 'time_hr': 768.0},
            {'name': 'San Francisco Bay', 'lat': 37.77, 'lon': -122.42, 'time_hr': 816.0},
            {'name': 'Panama Canal Entry', 'lat': 9.38, 'lon': -79.92, 'time_hr': 1104.0},
            {'name': 'Gulf of Mexico', 'lat': 25.00, 'lon': -90.00, 'time_hr': 1248.0},
            {'name': 'New Orleans Transfer', 'lat': 29.95, 'lon': -90.07, 'time_hr': 1296.0},
            {'name': 'Mississippi River System', 'lat': 35.00, 'lon': -90.00, 'time_hr': 1440.0},
            {'name': 'Illinois Waterway', 'lat': 40.00, 'lon': -88.00, 'time_hr': 1560.0},
            {'name': 'Chicago Port (End)', 'lat': 41.88, 'lon': -87.62, 'time_hr': 1608.0}
        ],
        'duration': (1500, 1550, 1450)
    },

    'VNHPH-USMIA': {
        'waypoints': [
            {'name': 'Hai Phong Port (Start)', 'lat': 20.86, 'lon': 106.68, 'time_hr': 0.0},
            {'name': 'Gulf of Tonkin Exit', 'lat': 19.50, 'lon': 108.50, 'time_hr': 12.0},
            {'name': 'South China Sea Central', 'lat': 15.00, 'lon': 112.00, 'time_hr': 48.0},
            {'name': 'Singapore Strait', 'lat': 1.27, 'lon': 103.83, 'time_hr': 120.0},
            {'name': 'Malacca Strait Exit', 'lat': 5.00, 'lon': 98.00, 'time_hr': 168.0},
            {'name': 'Indian Ocean Crossing', 'lat': 0.00, 'lon': 80.00, 'time_hr': 360.0},
            {'name': 'Gulf of Aden', 'lat': 12.00, 'lon': 45.00, 'time_hr': 480.0},
            {'name': 'Suez Canal Entry', 'lat': 29.93, 'lon': 32.55, 'time_hr': 540.0},
            {'name': 'Mediterranean Sea', 'lat': 35.00, 'lon': 20.00, 'time_hr': 600.0},
            {'name': 'Gibraltar Strait', 'lat': 36.00, 'lon': -5.00, 'time_hr': 660.0},
            {'name': 'Atlantic Ocean Crossing', 'lat': 30.00, 'lon': -40.00, 'time_hr': 840.0},
            {'name': 'Caribbean Sea Approach', 'lat': 20.00, 'lon': -75.00, 'time_hr': 960.0},
            {'name': 'Miami Port (End)', 'lat': 25.77, 'lon': -80.20, 'time_hr': 1008.0}
        ],
        'duration': (960, 1128)
    },

    'VNSGN-KRINC': {
        'waypoints': [
            {'name': 'Ho Chi Minh Port (Start)', 'lat': 10.77, 'lon': 106.71, 'time_hr': 0.0},
            {'name': 'Vung Tau Exit', 'lat': 10.33, 'lon': 107.07, 'time_hr': 2.2},
            {'name': 'South China Sea Entry', 'lat': 10.5, 'lon': 108.0, 'time_hr': 5.9},
            {'name': 'South China Sea Mid South', 'lat': 13.0, 'lon': 110.0, 'time_hr': 18.7},
            {'name': 'South China Sea Central', 'lat': 16.0, 'lon': 112.0, 'time_hr': 33.1},
            {'name': 'South China Sea North', 'lat': 19.0, 'lon': 113.5, 'time_hr': 46.6},
            {'name': 'Hainan East Approach', 'lat': 20.0, 'lon': 114.0, 'time_hr': 51.1},
            {'name': 'Taiwan Strait Entry', 'lat': 23.0, 'lon': 117.0, 'time_hr': 68.1},
            {'name': 'Taiwan Strait Mid', 'lat': 24.0, 'lon': 118.0, 'time_hr': 73.7},
            {'name': 'Taiwan Strait Exit', 'lat': 25.0, 'lon': 119.5, 'time_hr': 80.9},
            {'name': 'East China Sea South', 'lat': 27.0, 'lon': 121.0, 'time_hr': 90.9},
            {'name': 'East China Sea Mid', 'lat': 30.0, 'lon': 123.0, 'time_hr': 105.3},
            {'name': 'East China Sea North', 'lat': 33.0, 'lon': 126.0, 'time_hr': 122.3},
            {'name': 'Yellow Sea South', 'lat': 34.0, 'lon': 126.5, 'time_hr': 126.2},
            {'name': 'Yellow Sea Mid', 'lat': 35.0, 'lon': 126.0, 'time_hr': 131.2},
            {'name': 'Incheon Approach', 'lat': 37.0, 'lon': 126.5, 'time_hr': 141.2},
            {'name': 'Incheon Port (End)', 'lat': 37.46, 'lon': 126.64, 'time_hr': 143.3}
        ],
        'duration': (192, 240)
    },

    'VNDAD-KRPUS': {
        'waypoints': [
            {'name': 'Da Nang Port (Start)', 'lat': 16.05, 'lon': 108.22, 'time_hr': 0.0},
            {'name': 'Da Nang Channel Exit', 'lat': 15.8, 'lon': 108.5, 'time_hr': 1.3},
            {'name': 'South China Sea Entry', 'lat': 16.0, 'lon': 110.0, 'time_hr': 5.6},
            {'name': 'South China Sea Central', 'lat': 18.0, 'lon': 112.0, 'time_hr': 15.0},
            {'name': 'South China Sea North', 'lat': 21.0, 'lon': 113.0, 'time_hr': 25.0},
            {'name': 'Hainan Approach', 'lat': 20.0, 'lon': 110.5, 'time_hr': 23.1},
            {'name': 'Taiwan Strait Entry', 'lat': 23.0, 'lon': 117.0, 'time_hr': 40.6},
            {'name': 'Taiwan Strait Mid', 'lat': 24.0, 'lon': 118.0, 'time_hr': 46.2},
            {'name': 'Taiwan Strait Exit', 'lat': 25.0, 'lon': 119.5, 'time_hr': 53.4},
            {'name': 'East China Sea South', 'lat': 27.0, 'lon': 121.0, 'time_hr': 63.4},
            {'name': 'East China Sea Mid', 'lat': 30.0, 'lon': 123.0, 'time_hr': 77.8},
            {'name': 'East China Sea North', 'lat': 33.0, 'lon': 126.0, 'time_hr': 91.9},
            {'name': 'Busan Approach', 'lat': 34.5, 'lon': 128.0, 'time_hr': 101.9},
            {'name': 'Busan Port (End)', 'lat': 35.1, 'lon': 129.05, 'time_hr': 105.6}
        ],
        'duration': (120, 168)
    }
}


# 2. Cấu hình file, tham số API, cache (globals)

In [3]:
INPUT_GPS_FILE = r"C:\Users\Dell\Downloads\gps_trips_part2_2.csv"
OUTPUT_ENRICHED_FILE = "enriched_weather_data_2-2.csv"

OPEN_METEO_URL = "https://archive-api.open-meteo.com/v1/archive"
HOURLY_PARAMS = "temperature_2m,relative_humidity_2m,dew_point_2m,surface_pressure,wind_speed_10m,precipitation"

# Cache thời tiết (key: (date, roundlat, roundlon))
weather_cache = {}
api_call_count = 0
total_api_failures = 0

# Retry / throttle params (có thể chỉnh)
MAX_API_RETRIES = 5
BASE_SLEEP_SEC = 0.2


# 3. Hàm gọi API với cache + retry (get_weather_at_point)

In [4]:
def get_weather_at_point(timestamp, lat, lon):
    """
    Trả về dict với keys: temp, humidity, dew_point, pressure, wind_speed, precipitation
    Hoặc None nếu lỗi / không lấy được.
    """
    global api_call_count, total_api_failures, weather_cache

    # sanitize inputs
    try:
        current_time = pd.to_datetime(timestamp)
    except Exception:
        return None

    date_str = current_time.strftime('%Y-%m-%d')
    target_time_str = current_time.strftime('%Y-%m-%dT%H:00')

    query_lat = round(float(lat), 2)
    query_lon = round(float(lon), 2)
    cache_key = (date_str, query_lat, query_lon)

    # 1) Nếu đã cache trước đó
    if cache_key in weather_cache:
        hourly_data = weather_cache[cache_key]
        if not hourly_data:
            return None
    else:
        # 2) Chưa cache -> gọi API với retry
        params = {
            "latitude": query_lat,
            "longitude": query_lon,
            "start_date": date_str,
            "end_date": date_str,
            "hourly": HOURLY_PARAMS,
            "timezone": "UTC"
        }
        retry_count = 0
        while retry_count < MAX_API_RETRIES:
            try:
                resp = requests.get(OPEN_METEO_URL, params=params, timeout=30)
                api_call_count += 1
                if resp.status_code == 200:
                    data = resp.json()
                    # store hourly in cache (None nếu không có)
                    hourly = data.get('hourly')
                    weather_cache[cache_key] = hourly if hourly else None
                    time.sleep(BASE_SLEEP_SEC)
                    break
                elif resp.status_code == 429:
                    retry_count += 1
                    wait_time = 15 * retry_count
                    print(f"[WARN] API rate limit 429 for {cache_key}. Retrying after {wait_time}s ({retry_count}/{MAX_API_RETRIES})...")
                    time.sleep(wait_time)
                else:
                    print(f"[ERROR] API returned {resp.status_code} for {cache_key}: {resp.text[:200]}")
                    weather_cache[cache_key] = None
                    total_api_failures += 1
                    return None
            except requests.exceptions.RequestException as e:
                retry_count += 1
                wait_time = 5 * retry_count
                print(f"[NETWORK ERROR] {e}. Retry after {wait_time}s ({retry_count}/{MAX_API_RETRIES})...")
                time.sleep(wait_time)

        # sau vòng retry kiểm tra cache
        if cache_key not in weather_cache:
            print(f"[ERROR] Failed to fetch API for {cache_key} after retries.")
            weather_cache[cache_key] = None
            total_api_failures += 1
            return None

    # 3) Lấy dữ liệu từ cache
    hourly_data = weather_cache[cache_key]
    if not hourly_data:
        return None

    # Tìm index của giờ cần
    try:
        idx = hourly_data['time'].index(target_time_str)
        weather_at_hour = {
            'temp': hourly_data.get('temperature_2m', [None])[idx],
            'humidity': hourly_data.get('relative_humidity_2m', [None])[idx],
            'dew_point': hourly_data.get('dew_point_2m', [None])[idx],
            'pressure': hourly_data.get('surface_pressure', [None])[idx],
            'wind_speed': hourly_data.get('wind_speed_10m', [None])[idx],
            'precipitation': hourly_data.get('precipitation', [None])[idx],
        }
        return weather_at_hour
    except Exception:
        # time key missing or index error
        return None


# 4. Hàm xử lý (nội suy) 1 trip và trả về DataFrame đã enrich

In [5]:
def process_single_trip(trip_df, route_config):
    """
    Input:
      - trip_df: DataFrame chứa các điểm GPS cho một trip (cột tối thiểu: timestamp, latitude, longitude, trip_id, Route)
      - route_config: ROUTE_CONFIG dict
    Output:
      - new_trip_df: DataFrame copy với các cột thời tiết nội suy (temperature_C, humidity_%, dew_point_C, pressure_hPa, wind_speed_kmh, precipitation_mm)
    """
    # Sao chép để an toàn
    new_trip_df = trip_df.copy()

    # Chuẩn hóa timestamp: convert -> tz-naive (để khỏi so sánh tz-aware vs tz-naive)
    new_trip_df['timestamp'] = pd.to_datetime(new_trip_df['timestamp'], errors='coerce')
    new_trip_df = new_trip_df.dropna(subset=['timestamp'])
    if new_trip_df.empty:
        return None

    # Lấy route key và cấu hình
    route_key = new_trip_df['Route'].iloc[0].strip()
    config = route_config.get(route_key)
    if not config:
        # Không có cấu hình route -> bỏ
        return None

    # Lấy các giờ neo (key hours)
    route_waypoints = config['waypoints']
    KEY_HOURS = [wp['time_hr'] for wp in route_waypoints if 'time_hr' in wp]

    if len(KEY_HOURS) < 2:
        # Không đủ điểm neo để nội suy
        # tạo các cột nhưng fill NaN
        for col in ['temperature_C','humidity_%','dew_point_C','pressure_hPa','wind_speed_kmh','precipitation_mm']:
            new_trip_df[col] = np.nan
        return new_trip_df

    # T0 = thời gian bắt đầu trip
    start_time_trip = new_trip_df['timestamp'].min()

    # Thông tin neo lấy từ GPS gần nhất tới các mốc giờ (start_time + base_hour)
    key_weather_frames = {'timestamp_hours': [], 'temp': [], 'humidity': [], 'dew_point': [], 'pressure': [], 'wind_speed': [], 'precipitation': []}

    for base_hour in KEY_HOURS:
        target_timestamp = start_time_trip + pd.Timedelta(hours=float(base_hour))
        # tìm điểm GPS gần nhất về thời gian
        # dùng idxmin trên abs difference
        diffs = (new_trip_df['timestamp'] - target_timestamp).abs()
        idx_min = diffs.idxmin()
        row = new_trip_df.loc[idx_min]
        lat = row['latitude']
        lon = row['longitude']
        timestamp_for_api = row['timestamp']

        # lấy weather
        w = get_weather_at_point(timestamp_for_api, lat, lon)
        if w:
            actual_hour_rel = (timestamp_for_api - start_time_trip).total_seconds() / 3600.0
            key_weather_frames['timestamp_hours'].append(actual_hour_rel)
            key_weather_frames['temp'].append(w['temp'])
            key_weather_frames['humidity'].append(w['humidity'])
            key_weather_frames['dew_point'].append(w['dew_point'])
            key_weather_frames['pressure'].append(w['pressure'])
            key_weather_frames['wind_speed'].append(w['wind_speed'])
            key_weather_frames['precipitation'].append(w['precipitation'])

    # Nếu không có dữ liệu weather đủ -> gán NaN
    if len(key_weather_frames['timestamp_hours']) < 2:
        for col in ['temperature_C','humidity_%','dew_point_C','pressure_hPa','wind_speed_kmh','precipitation_mm']:
            new_trip_df[col] = np.nan
        return new_trip_df

    # Chuẩn bị trục X (giờ) cho nội suy
    new_trip_df['timestamp_hours'] = (new_trip_df['timestamp'] - start_time_trip).dt.total_seconds() / 3600.0
    target_hours = new_trip_df['timestamp_hours'].values

    # Sắp xếp các điểm neo theo giờ
    known_hours = np.array(key_weather_frames['timestamp_hours'])
    order = np.argsort(known_hours)
    known_hours_sorted = known_hours[order]

    # Nội suy cho từng tham số
    param_map = {
        'temp': 'temperature_C',
        'humidity': 'humidity_%',
        'dew_point': 'dew_point_C',
        'pressure': 'pressure_hPa',
        'wind_speed': 'wind_speed_kmh',
        'precipitation': 'precipitation_mm'
    }

    for src_key, out_col in param_map.items():
        values = np.array(key_weather_frames[src_key])[order]
        try:
            interp_vals = np.interp(target_hours, known_hours_sorted, values)
            new_trip_df[out_col] = interp_vals
        except Exception:
            new_trip_df[out_col] = np.nan

    # Xóa cột tạm
    new_trip_df.drop(columns=['timestamp_hours'], inplace=True, errors='ignore')

    return new_trip_df


# 5. Chuẩn bị input file, kiểm tra và tạo trạng thái resume/append

In [6]:
print("Đang đọc file GPS input:", INPUT_GPS_FILE)
try:
    df_gps = pd.read_csv(INPUT_GPS_FILE, parse_dates=['timestamp'])
except FileNotFoundError:
    raise FileNotFoundError(f"Không tìm thấy file GPS input: {INPUT_GPS_FILE}")
except Exception as e:
    raise RuntimeError(f"Lỗi đọc file GPS input: {e}")

print("Số hàng GPS đọc được:", len(df_gps))

# Chuẩn hóa: xoá khoảng trắng trong Route, chuyển timestamp -> tz-naive
if 'Route' not in df_gps.columns:
    raise RuntimeError("File GPS không có cột 'Route'. Vui lòng kiểm tra đầu vào.")

df_gps['Route'] = df_gps['Route'].astype(str).str.replace(" ", "").str.strip()
df_gps['timestamp'] = pd.to_datetime(df_gps['timestamp'], errors='coerce')
df_gps = df_gps.dropna(subset=['timestamp']).copy()

# Kiểm tra file output để resume nếu cần
processed_trip_ids = set()
file_exists = os.path.exists(OUTPUT_ENRICHED_FILE)
is_first_trip = True
if file_exists:
    try:
        df_existing = pd.read_csv(OUTPUT_ENRICHED_FILE, usecols=['trip_id'])
        processed_trip_ids = set(df_existing['trip_id'].unique())
        is_first_trip = False
        print(f"Phát hiện {len(processed_trip_ids)} trip đã xử lý (sẽ bỏ qua).")
    except Exception as e:
        print(f"Cảnh báo: không đọc được file output cũ (sẽ viết mới). Lỗi: {e}")
        is_first_trip = True


Đang đọc file GPS input: C:\Users\Dell\Downloads\gps_trips_part2_2.csv
Số hàng GPS đọc được: 734320
Phát hiện 347 trip đã xử lý (sẽ bỏ qua).


# 6. Vòng lặp xử lý từng trip và ghi append file 

In [7]:
start_time_total = time.time()
total_points_processed = 0
new_trips_processed = 0

df_gps_grouped = df_gps.groupby('trip_id')
print("Tổng chuyến cần xử lý:", len(df_gps_grouped))

for trip_id, trip_df in tqdm(df_gps_grouped, total=len(df_gps_grouped), desc="Xử lý chuyến"):
    # Bỏ qua trip đã xử lý trước đó
    if trip_id in processed_trip_ids:
        continue

    # Gọi hàm xử lý 1 trip
    enriched_trip = process_single_trip(trip_df, ROUTE_CONFIG)
    if enriched_trip is None:
        # Bỏ qua nếu không có cấu hình route hoặc lỗi
        continue

    # GHI/APPEND vào file output
    if is_first_trip:
        enriched_trip.to_csv(OUTPUT_ENRICHED_FILE, index=False, mode='w', header=True, encoding='utf-8-sig')
        is_first_trip = False
    else:
        enriched_trip.to_csv(OUTPUT_ENRICHED_FILE, index=False, mode='a', header=False, encoding='utf-8-sig')

    total_points_processed += len(enriched_trip)
    new_trips_processed += 1

    # (tùy chọn) cập nhật processed_trip_ids để bỏ qua trong lần chạy cùng session
    processed_trip_ids.add(trip_id)

end_time_total = time.time()
dur_minutes = (end_time_total - start_time_total) / 60.0
print(f"\nHoàn tất. Thời gian chạy: {dur_minutes:.2f} phút.")
print(f"Đã xử lý {new_trips_processed} chuyến mới, tổng điểm lưu: {total_points_processed}.")
print(f"File output: {OUTPUT_ENRICHED_FILE}")
print(f"Tổng API calls (ước tính): {api_call_count}, total_api_failures: {total_api_failures}")


Tổng chuyến cần xử lý: 1500


Xử lý chuyến:  23%|██▎       | 352/1500 [01:04<06:07,  3.12it/s]

[NETWORK ERROR] ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)). Retry after 5s (1/5)...


Xử lý chuyến:  24%|██▍       | 362/1500 [03:28<1:57:19,  6.19s/it]

[NETWORK ERROR] ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)). Retry after 5s (1/5)...


...

[WARN] API rate limit 429 for ('2024-02-28', 15.55, 111.69). Retrying after 15s (1/5)...
[WARN] API rate limit 429 for ('2024-02-28', 15.55, 111.69). Retrying after 30s (2/5)...
[WARN] API rate limit 429 for ('2024-02-28', 15.55, 111.69). Retrying after 45s (3/5)...
[WARN] API rate limit 429 for ('2024-02-28', 15.55, 111.69). Retrying after 60s (4/5)...
[WARN] API rate limit 429 for ('2024-02-28', 15.55, 111.69). Retrying after 75s (5/5)...
[ERROR] Failed to fetch API for ('2024-02-28', 15.55, 111.69) after retries.
[WARN] API rate limit 429 for ('2024-02-29', 18.25, 112.57). Retrying after 15s (1/5)...
[WARN] API rate limit 429 for ('2024-02-29', 18.25, 112.57). Retrying after 30s (2/5)...
[WARN] API rate limit 429 for ('2024-02-29', 18.25, 112.57). Retrying after 45s (3/5)...
[WARN] API rate limit 429 for ('2024-02-29', 18.25, 112.57). Retrying after 60s (4/5)...


Xử lý chuyến:  98%|█████████▊| 1476/1500 [4:59:23<06:09, 15.39s/it]   

[NETWORK ERROR] HTTPSConnectionPool(host='archive-api.open-meteo.com', port=443): Read timed out. (read timeout=30). Retry after 5s (1/5)...


Xử lý chuyến: 100%|██████████| 1500/1500 [5:04:27<00:00, 12.18s/it]


Hoàn tất. Thời gian chạy: 304.47 phút.
Đã xử lý 1153 chuyến mới, tổng điểm lưu: 576064.
File output: enriched_weather_data_2-2.csv
Tổng API calls (ước tính): 12771, total_api_failures: 1


# Các cell sau gộp data dành cho bảng 3

In [ ]:
# Kiểm tra file output hiện tại 
from pathlib import Path
import pandas as pd
OUTPUT_ENRICHED_FILE = "enriched_weather_data.csv"  

p = Path(OUTPUT_ENRICHED_FILE)
if not p.exists():
    print("Không tìm thấy file:", OUTPUT_ENRICHED_FILE)
else:
    print("Tồn tại:", p.resolve())
    # đọc 5 dòng đầu và 5 dòng cuối để kiểm tra
    df_head = pd.read_csv(p, nrows=5)
    print("=== 5 dòng đầu ===")
    display(df_head)
    # đọc 5 dòng cuối (đọc bằng chunk để tiết kiệm mem)
    tail_rows = pd.read_csv(p, skiprows=max(1, sum(1 for _ in open(p))-6))
    print("=== 5 dòng cuối (nếu đọc được) ===")
    display(tail_rows.head())


Tồn tại: C:\Users\Dell\Downloads\enriched_weather_data.csv
=== 5 dòng đầu ===


,trip_id,fruit_cate,timestamp,latitude,longitude,speed_knots,Route,temperature_C,humidity_%,dew_point_C,pressure_hPa,wind_speed_kmh,precipitation_mm
0,TRIP_28773,Tomato,2024-01-24 20:37:07,10.753420,106.692127,0.00,VNSGN-USCHI,24.9,75.0,20.1,1014.60,4.300,0.0
1,TRIP_28773,Tomato,2024-01-24 21:37:07,10.625561,106.811287,10.41,VNSGN-USCHI,24.8,73.5,19.7,1014.95,8.025,0.0
2,TRIP_28773,Tomato,2024-01-24 22:37:07,10.500961,106.914143,9.63,VNSGN-USCHI,24.7,72.0,19.3,1015.30,11.750,0.0
3,TRIP_28773,Tomato,2024-01-24 23:37:07,10.384252,107.026470,9.65,VNSGN-USCHI,24.6,70.5,18.9,1015.65,15.475,0.0
4,TRIP_28773,Tomato,2024-01-25 00:37:07,10.355797,107.158500,7.98,VNSGN-USCHI,24.5,69.0,18.5,1016.00,19.200,0.0


=== 5 dòng cuối (nếu đọc được) ===


,TRIP_77982,Pineapple,2024-01-15 06:04:13,36.759389730546765,126.44053713346302,8.51,VNSGN-KRINC,27.0,79.0,23.1,1011.1,36.8,0.0
0,TRIP_77982,Pineapple,2024-01-15 07:04:13,36.875428,126.476195,7.17,VNSGN-KRINC,27.0,79.0,23.1,1011.1,36.8,0.0
1,TRIP_77982,Pineapple,2024-01-15 08:04:13,37.018330,126.500618,8.66,VNSGN-KRINC,27.0,79.0,23.1,1011.1,36.8,0.0
2,TRIP_77982,Pineapple,2024-01-15 09:04:13,37.156358,126.554452,8.68,VNSGN-KRINC,27.0,79.0,23.1,1011.1,36.8,0.0
3,TRIP_77982,Pineapple,2024-01-15 10:04:13,37.299810,126.592670,8.80,VNSGN-KRINC,27.0,79.0,23.1,1011.1,36.8,0.0
4,TRIP_77982,Pineapple,2024-01-15 11:04:13,37.434378,126.629756,8.27,VNSGN-KRINC,27.0,79.0,23.1,1011.1,36.8,0.0


In [1]:
import pandas as pd
from pathlib import Path

# 🔧 Bước 1: Khai báo danh sách file cần gộp
file_paths = [
    r"C:\Users\Dell\Downloads\enriched_weather_data.csv",
    r"C:\Users\Dell\Downloads\enriched_weather_data1468.csv",
    r"C:\Users\Dell\Downloads\enriched_weather_data500.csv"
]

# 🔧 Bước 2: Đọc & gộp
dfs = []
for f in file_paths:
    p = Path(f)
    if not p.exists():
        print(f"⚠️ Không tìm thấy file: {f}")
        continue
    print(f"Đang đọc file: {f}")
    df = pd.read_csv(f)
    dfs.append(df)

# 🔧 Bước 3: Gộp tất cả vào một DataFrame duy nhất
if len(dfs) == 0:
    raise ValueError("Không có file CSV hợp lệ để gộp!")

df_merged = pd.concat(dfs, ignore_index=True)
print(f"✅ Đã gộp {len(dfs)} file, tổng số dòng: {len(df_merged):,}")


# 🔧 Bước 5: Lưu ra file mới (CSV & Parquet)
output_csv = r"C:\Users\Dell\Downloads\enriched_weather_data_merged.csv"
output_parquet = r"C:\Users\Dell\Downloads\enriched_weather_data_merged.parquet"

df_merged.to_csv(output_csv, index=False, encoding='utf-8-sig')
df_merged.to_parquet(output_parquet, index=False)

print(f"💾 Đã lưu:\n - CSV: {output_csv}\n - Parquet: {output_parquet}")


Đang đọc file: C:\Users\Dell\Downloads\enriched_weather_data.csv
Đang đọc file: C:\Users\Dell\Downloads\enriched_weather_data1468.csv
Đang đọc file: C:\Users\Dell\Downloads\enriched_weather_data500.csv
✅ Đã gộp 3 file, tổng số dòng: 1,829,459
💾 Đã lưu:
 - CSV: C:\Users\Dell\Downloads\enriched_weather_data_merged.csv
 - Parquet: C:\Users\Dell\Downloads\enriched_weather_data_merged.parquet


In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
input_path = Path(r"C:\Users\Dell\Downloads\Iot.csv")
df = pd.read_csv(input_path)
print("Kích thước ban đầu:", df.shape)
print("Các cột:", df.columns.tolist())
display(df.head(5))

Kích thước ban đầu: (10958389, 19)
Các cột: ['timestamp', 'temp', 'humid', 'co2', 'light', 'class', 'trip_id', 'fruit_cate', 'latitude', 'longitude', 'speed_knots', 'Route', 'temperature_C', 'humidity_%', 'dew_point_C', 'pressure_hPa', 'wind_speed_kmh', 'precipitation_mm', 'failure_scenario']


,timestamp,temp,humid,co2,light,class,trip_id,fruit_cate,latitude,longitude,speed_knots,Route,temperature_C,humidity_%,dew_point_C,pressure_hPa,wind_speed_kmh,precipitation_mm,failure_scenario
0,2024-01-21 19:02:58,21.944977,85.257717,310.470178,14.473861,Good,TRIP_00002,Pineapple,20.844783,106.76401,0.0,VNHPH-CNSHA,16.4,87.0,14.2,1017.2,24.4,0.0,GOOD
1,2024-01-21 19:12:58,21.930127,84.937926,310.754231,11.989037,Good,TRIP_00002,Pineapple,20.844783,106.76401,0.0,VNHPH-CNSHA,16.4,87.0,14.2,1017.2,24.4,0.0,GOOD
2,2024-01-21 19:22:58,22.082836,84.535408,311.071613,14.079832,Good,TRIP_00002,Pineapple,20.844783,106.76401,0.0,VNHPH-CNSHA,16.4,87.0,14.2,1017.2,24.4,0.0,GOOD
3,2024-01-21 19:32:58,22.067485,84.786104,311.393199,13.468183,Good,TRIP_00002,Pineapple,20.844783,106.76401,0.0,VNHPH-CNSHA,16.4,87.0,14.2,1017.2,24.4,0.0,GOOD
4,2024-01-21 19:42:58,22.109032,84.616012,311.723067,14.142465,Good,TRIP_00002,Pineapple,20.844783,106.76401,0.0,VNHPH-CNSHA,16.4,87.0,14.2,1017.2,24.4,0.0,GOOD
